# Introduction

### What is Employee Attrition

Employee attrition, especially those of key and star employees is a major concern for an HR organization. When employees leave, it has many side effects. There is loss of organizational and product-specific expertise, loss of productivity due to new hires taking time to onboard. Sometimes employees have great relationships with customers, and that is hard to rebuild. There are also hiring costs and training costs associated. Employees leave due to various reasons.

They include compensation, work satisfaction, performance, and issues with their supervisors. The online world had made it easy for outside recruiters to approach employees with better job offers, which can make a content employee leave.

### How can AI help in this employee attrition?

First, we need to collect 360-degree data about the employee's past and present. This include, but not limited to the tenure in the company, performance ratings, compensation and promotions. Relationships the employee has with their supervisor and peers also play a key part. 360 reviews will help understand this. Once the data is collected and associated with both past and present employees, it provides input to build an ML model to predict attrition. Then, HR can take preventive action that is needed.

# Classification Using Deep Learning

Let's quickly review the concept of classification in machine learning and its implementation in deep learning. Classification is a machine learning problem of identifying a set of categories to which a new observation belongs based on a training dataset that contains observations for which the category is already known. Classification is the most common type of machine learning problem.

Examples include binary classification, like if a customer would buy a product or not, or multiclass classification, like determining the type of customer based on customer demographics. There are also a number of libraries that provide implementation of these algorithms. In Python, the most popular one is scikit-learn.

In recent years, deep learning has revolutionized classification with its ability to handle complex relationships and generate highly accurate models. If you're not familiar with deep learning, I recommend watching other courses on this website that explains the basics of deep learning.

We will use Keras library with TensorFlow backend for building the classification model.

# About The Dataset

The dataset contains feature attributes that could possibly impact an employee's decision to leave the organization.

The first column is employee ID.

The second column is the total months of experience that the employee has overall.

Total orgs worked is the total number of organizations the employee has worked so far.

Months in org is the total number of months the employee has worked in this organization.

Last pay increment band is a scale of one to five with one being the highest. The higher the band, the higher the pay raise for the employee in the last increment cycle.

Next comes average feedback based on the 360 degree feedback the employee got in performance reviews. Again, it's a scale of one to five, with one being the highest rating.

Last promotion years represent the total number of years since the last promotion for the employee.

Finally, we have the target variable attrition, which indicates if the employee left the organization or not. This is only a representational data set. For your organization, you should do enough research to include all kinds of data that may possibly influence an employee's decision.

# Preprocessing Dataset

We first load the data using pandas into a data frame, then we review the data loaded, its structure and its contents. We can see that the data has been loaded correctly. In classification, it's always a good idea to understand the relationship between the feature and the target variables, especially which feature variables have the most impact on the target variable. We do so using correlation analysis. Here, we do a correlation analysis on the target variable, attrition.

We see that the LastPromotionYears has a significant impact on attrition, meaning that employees leave when they don't see enough career growth.

Next, we prepare the data for machine learning. We first convert the dataset into a NumPy array of type float. This is the preferred input format for Keras.

Next, we split the feature and the target variables into X and Y. We leave out the EmployeeID. We could additionally do center and scaling too, if the accuracy is too low. For the target variable, we will use One-Hot Encoding using the Keras to_categorical function. Since the attrition is Boolean, it has two unique values.

Finally, we print the shapes on X and Y. Let's run this code and review the results. We see that there are a thousand samples. X has six columns for the six attributes. Y has two columns since it has One-Hot Encoding for two unique values. In the next video, we will build a model for attrition.

In [2]:
#Load the dataset and analyze

import pandas as pd
import os
import tensorflow as tf
import numpy as np

attrition_data = pd.read_csv("employee_attrition.csv")

print("Data Loaded:\n------------------------\n",attrition_data.dtypes)
attrition_data.head()
attrition_data.shape

Data Loaded:
------------------------
 EmployeeID              int64
TotalMonthsOfExp        int64
TotalOrgsWorked         int64
MonthsInOrg             int64
LastPayIncrementBand    int64
AverageFeedback         int64
LastPromotionYears      int64
Attrition               int64
dtype: object


(1000, 8)

In [3]:
attrition_data.describe()

,EmployeeID,TotalMonthsOfExp,TotalOrgsWorked,MonthsInOrg,LastPayIncrementBand,AverageFeedback,LastPromotionYears,Attrition
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,61.002000,3.056000,31.000000,3.063000,2.539000,2.45400,0.230000
std,288.819436,34.818132,1.398148,26.783341,1.413162,1.143585,1.10413,0.421043
min,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.00000,0.000000
25%,250.750000,31.000000,2.000000,9.000000,2.000000,1.000000,1.00000,0.000000
50%,500.500000,61.000000,3.000000,23.000000,3.000000,3.000000,2.00000,0.000000
75%,750.250000,91.250000,4.000000,48.000000,4.000000,4.000000,3.00000,0.000000
max,1000.000000,120.000000,5.000000,116.000000,5.000000,4.000000,4.00000,1.000000


In [4]:
#Correlation Analysis of target attribute

attrition_data.corr()['Attrition']

EmployeeID             -0.036630
TotalMonthsOfExp        0.019702
TotalOrgsWorked         0.008706
MonthsInOrg             0.012605
LastPayIncrementBand    0.108528
AverageFeedback        -0.008253
LastPromotionYears      0.765641
Attrition               1.000000
Name: Attrition, dtype: float64

In [5]:
#Convert to Dataframe to numpy array
np_attrition = attrition_data.to_numpy().astype(float)

#Create X_train with the first 7 attributes
X_train = np_attrition[:,1:7]
#Create Y_train with attrition attribute
Y_train=np_attrition[:,7]

#Convert Y_train to one-hot-encoding
Y_train = tf.keras.utils.to_categorical(Y_train,2)

print("X-Train Shape : ", X_train.shape)
print("Y-Train Shape : ", Y_train.shape)

X-Train Shape :  (1000, 6)
Y-Train Shape :  (1000, 2)


# Building Attrition Model With Keras

We start off by setting up the hyper parameters for the model. We will use an epoch of 100 and a batch size of 100. We will set verbose to one so that we can see the training results. The number of unique output classes is two. We will create hidden layers for 128 units. We will also do a validation split of 0.2. This means 20% of the data will be used for validation while building the model. I highly recommend experimenting with the hyper parameters to understand how they impact model accuracy.

Now, we can create a Keras model. Then we add a dense, hidden layer with ReLU as activation. We then add a second hidden layer. Finally, we add an output dense layer with softmax activation. We compiled the model with Adam optimizer and use categorical crossentropy as the last function. We also measure the accuracy of the model. Then we fit the model for this input data. This set of steps is a standard template for building basic deep learning models.We can see that the model ended up with an accuracy of 99%.

In [6]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Setup hyperparameters for deep learning
EPOCHS=100
BATCH_SIZE=100
VERBOSE=1
NB_CLASSES=2
N_HIDDEN=128
VALIDATION_SPLIT=0.2

#Create a Keras model
model = tf.keras.models.Sequential()

#Add first hidden Dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(6,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second hidden dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a final layer with softmax
model.add(keras.layers.Dense(NB_CLASSES,
                             name='Final',
                             activation='softmax'))

#Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Fit parameters
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

Epoch 1/100
8/8 [==============================] - 1s 35ms/step - loss: 1.0907 - accuracy: 0.6538 - val_loss: 0.7323 - val_accuracy: 0.6000
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6519 - accuracy: 0.7275 - val_loss: 0.5775 - val_accuracy: 0.7650
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 0.5594 - accuracy: 0.7638 - val_loss: 0.5637 - val_accuracy: 0.7850
Epoch 4/100
8/8 [==============================] - 0s 7ms/step - loss: 0.5279 - accuracy: 0.7688 - val_loss: 0.5161 - val_accuracy: 0.7800
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 0.5039 - accuracy: 0.7800 - val_loss: 0.4816 - val_accuracy: 0.7850
Epoch 6/100
8/8 [==============================] - 0s 10ms/step - loss: 0.4699 - accuracy: 0.7950 - val_loss: 0.4890 - val_accuracy: 0.7800
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 0.4442 - accuracy: 0.8163 - val_loss: 0.4432 - val_accuracy: 0.8000
Epoch 8/100
8/8 [========

In [7]:
from tensorflow.keras.models import save_model

# Save the model to an HDF5 file
model.save('your_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Predicting Attrition With Keras

In order to predict attrition, we need to get the same set of six feature variables about the new employee for whom we need to predict attrition. Prediction then is an easy call to the model.predict_classes method that will output a zero or one based on the features.

We can see that the algorithm predicts that the employee will leave. We can also do bulk predictions for multiple employees. We need to create an array of arrays with the inner array representing an employee. The method call is the same.

I would strongly recommend using additional data points as well as experimenting with hyperparameters and layers for your own model.

In [8]:
import pickle

# Save the model using pickle
with open('your_model1.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
TotalMonthsOfExp=40
TotalOrgsWorked=4
MonthsInOrg=20
LastPayIncrementBand=5
AverageFeedback=4
LastPromotionYears=4

print("Will employee leave ?")

prediction= np.argmax(model.predict([[TotalMonthsOfExp,
                                  TotalOrgsWorked,
                                  MonthsInOrg,
                                  LastPayIncrementBand,
                                  AverageFeedback,
                                  LastPromotionYears]]), axis=1)
print(prediction)

Will employee leave ?
1/1 [==============================] - 0s 103ms/step
[1]


In [ ]:
## PREDICTION: ATTRITION PERCENTAGE

TotalMonthsOfExp=40
TotalOrgsWorked=4
MonthsInOrg=20
LastPayIncrementBand=5
AverageFeedback=4
LastPromotionYears=4

print("Will employee leave ?")

prediction= model.predict([[TotalMonthsOfExp,
                                  TotalOrgsWorked,
                                  MonthsInOrg,
                                  LastPayIncrementBand,
                                  AverageFeedback,
                                  LastPromotionYears]])
print(round(prediction[0][1]*100,2))

Will employee leave ?
1/1 [==============================] - 0s 37ms/step
91.14


In [ ]:
## TESTING

TotalMonthsOfExp=80
TotalOrgsWorked=2
MonthsInOrg=40
LastPayIncrementBand=5
AverageFeedback=4
LastPromotionYears=4

print("Will employee leave ?")

prediction= model.predict([[TotalMonthsOfExp,
                                  TotalOrgsWorked,
                                  MonthsInOrg,
                                  LastPayIncrementBand,
                                  AverageFeedback,
                                  LastPromotionYears]])
print(round(prediction[0][1]*100,2))

Will employee leave ?
1/1 [==============================] - 0s 38ms/step
84.26


In [ ]:
#Bulk predictions

bulk_pred=np.argmax(model.predict(
    [[111,5,85,3,2,2],
    [31,2,15,4,1,4],
    [61,4,24,1,4,3],
    [77,4,35,3,1,1],
    [81,5,7,1,2,3],
    [113,4,112,5,4,1],
    [101,2,48,5,1,4],
    [45,4,22,5,3,1],
    [25,2,2,2,3,2],
    [97,3,15,3,2,4]]), axis=1)

bulk_pred

1/1 [==============================] - 0s 37ms/step


array([0, 1, 0, 0, 0, 0, 1, 0, 0, 1])